# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [1]:
import torch as th
from torch.utils.data import Dataset
from sklearn.datasets import make_regression

In [2]:
X, y = make_regression(n_samples=1000, n_features=10)
X.shape, y.shape, type(X)

((1000, 10), (1000,), numpy.ndarray)

In [3]:
from typing import Callable

class RegressionDataset(Dataset):
    def __init__(self, transform: Callable | None = None, **kwargs):
        super().__init__()
        self.X, self.y = make_regression(**kwargs)
        self.transform = transform

    def custom_method(self):
        ...

    def __getitem__(self, idx):
        x = self.X[idx]
        if self.transform is not None:
            x = self.transform(x)
        y = self.y[idx]

        return x, y

    def __len__(self):
        return len(self.X)

In [4]:
def f(x):
    return x
f(5)

5

In [5]:
class MyCallable:
    def __call__(self, x):
        return x

In [6]:
c = MyCallable()
c(5)

5

In [7]:
dataset = RegressionDataset(n_samples=1000, n_features=10)
dataset[0]

(array([ 0.99217507, -1.69622309, -1.52175968,  0.68054118, -0.05119466,
        -1.99717461, -3.23973195,  2.02409894, -0.59440068,  1.29610266]),
 np.float64(-136.6350778713707))

In [8]:
import numpy as np

def my_transformer(x: np.ndarray) -> np.ndarray:
    return 1000 * x

In [9]:
class MyCallable:
    def __init__(self, coef: int) -> None:
        self._coef = coef

    def __call__(self, x: np.ndarray) -> np.ndarray:
        return self._coef * x

In [10]:
dataset = RegressionDataset(
        transform=my_transformer,
        n_samples=1000,
        n_features=10
)
dataset[0]

(array([ -248.98954205, -1492.48816183,  -629.35497785,  -775.610547  ,
        -1680.48579853,  -313.16479768,  -904.02834667,    30.69329618,
          796.88664468,  1488.55391957]),
 np.float64(-260.5798618513616))

In [11]:
dataset = RegressionDataset(
        transform=MyCallable(coef=10000),
        n_samples=1000,
        n_features=10
)
dataset[0]

(array([  5264.81787076,  16691.02317881,   -281.27433583,   2277.41529903,
         -1838.66935958, -21189.90619873,   3690.05172596,  -8946.94250598,
          6347.58672713,  10819.70532517]),
 np.float64(-7.886814365504748))

In [12]:
dataset[:2]

(array([[  5264.81787076,  16691.02317881,   -281.27433583,
           2277.41529903,  -1838.66935958, -21189.90619873,
           3690.05172596,  -8946.94250598,   6347.58672713,
          10819.70532517],
        [  7049.06680366, -11327.3725911 ,   4271.48465986,
           3456.45235385,   3276.90552455,   3069.94178126,
          -5911.22251993,  13697.14888583,   3484.50248534,
          -6766.00940918]]),
 array([-7.88681437, 54.56685176]))

In [13]:
from torch.utils.data import random_split

In [14]:
train, val, test = random_split(dataset, lengths=[0.7, 0.15, 0.15])

In [15]:
train[0]

(array([ -1729.47784406,  -4667.94067917, -15524.60012856,   4032.24697736,
         17094.57690716,  11036.05452279,  -6681.33751414, -10187.68429732,
         -8517.40663239,   2844.61857957]),
 np.float64(119.80183888473545))

In [16]:
len(train)

700

In [17]:
from torch.utils.data import DataLoader

In [18]:
loader = DataLoader(train, 64, )

it = iter(loader)
x, y = next(it)
x.shape, y.shape

(torch.Size([64, 10]), torch.Size([64]))

In [19]:
for x, y in loader:
    print(x.shape, y.shape)
    # break

torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([60, 10]) torch.Size([60])


In [20]:
700 % 64

60

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`.

Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` (серию) и `i`-ую строку из `obj.y` (строку).
        
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

- [ ] Проверено на семинаре

In [21]:
import pandas as pd

In [22]:
data = pd.read_csv('./../data/bank-full.csv')
data.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no


In [23]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        self.data = data
        self.X, self.y = data.drop(columns='y'), data['y']

    def __getitem__(self, idx: int) -> tuple:
        x = self.X.iloc[idx]
        y = self.y.iloc[idx]
        return x, y

    def __len__(self) -> int:
        return self.X.shape[0]

In [24]:
bank_dataset = BankDatasetBase(data)
bank_dataset[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

<p class="task" id="2"></p>

2\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательные аргументы `transform` и `target_transform`;
* если аргумент `transform` был передан, то при получении `i`-го элемента, нужно вызвать `transform(x)` и вернуть полученный результат.
* если аргумент `target_transform` был передан, то при получении `i`-го элемента, нужно вызвать `target_transform(y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность (без передачи `target_transform` и `transform`).

- [ ] Проверено на семинаре

In [25]:
class BankDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        transform: Callable | None = None,
        target_transform: Callable | None = None
    ) -> None:
        super().__init__()
        self.data = data
        self.X, self.y = data.drop(columns='y'), data['y']
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx: int) -> tuple:
        x = self.X.iloc[idx]
        y = self.y.iloc[idx]
        if self.transform is not None:
            x, y = self.transform(x, y)
        if self.target_transform is not None:
            x, y = self.target_transform(x, y)
        return x, y

    def __len__(self) -> int:
        return self.X.shape[0]

In [26]:
bank_dataset = BankDataset(data)
bank_dataset[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

<p class="task" id="3"></p>

3\. Опишите класс `OrdinalEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (признаки) и возвращает набор признаков, в котором нечисловые характеристики закодированы целыми числами;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
        
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `OrdinalEncoderTransform`.

- [ ] Проверено на семинаре

In [27]:
category_columns = data.select_dtypes('object').columns.tolist()
category_columns

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome',
 'y']

In [28]:
class Transform:
    def __init__(self, columns=[]):
        self.columns = columns
        self.encoders = {col: {} for col in columns}

In [29]:
class OrdinalEncoderTransform(Transform):
    def __init__(self, category_columns: list[str]) -> None:
        super().__init__(category_columns)

    def __call__(self, x_: pd.Series, y_: str) -> pd.Series:
        x = x_.copy()
        for col in self.columns:
            if col in x:
                if x[col] not in self.encoders[col]:
                    self.encoders[col][x[col]] = len(self.encoders[col])
                x.at[col] = self.encoders[col][x[col]]
        return x, y

In [30]:
transform = OrdinalEncoderTransform(category_columns)
bank_dataset = BankDataset(data, transform=transform)
x, y = bank_dataset[0]
x

age            58
job             0
marital         0
education       0
default         0
balance      2143
housing         0
loan            0
contact         0
day             5
month           0
duration      261
campaign        1
pdays          -1
previous        0
poutcome        0
Name: 0, dtype: object

In [31]:
x, y = bank_dataset[20]
x

age           28
job            1
marital        0
education      1
default        0
balance      723
housing        0
loan           1
contact        0
day            5
month          0
duration     262
campaign       1
pdays         -1
previous       0
poutcome       0
Name: 20, dtype: object

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (строку) и возвращает целое число, соответствующее этой строке;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
        
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объекта в качестве аргумента `target_transform` объект класса `LabelEncoderTransform`.

- [ ] Проверено на семинаре

In [32]:
class LabelEncoderTransform(Transform):
    def __init__(self) -> None:
        super().__init__()

    def __call__(self, x: pd.Series, y: str) -> int:
        if y not in self.encoders:
            self.encoders[y] = len(self.encoders)
        y = self.encoders[y]

        return x, y

In [33]:
transform = OrdinalEncoderTransform(category_columns)
label_encoder = LabelEncoderTransform()
bank_dataset = BankDataset(data, transform=transform, target_transform=label_encoder)
x, y = bank_dataset[0]
y

0

In [34]:
x, y = bank_dataset[83]
y

1

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.    Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает на вход серию или фрейм и возвращает тензор.

Опишите класс `Compose`.    Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании преобразования `Compose` список из объектов LabelEncoderTransform и ToTensor.

- [ ] Проверено на семинаре

In [35]:
from typing import Any

In [36]:
class ToTensor:
    def __call__(self, X: pd.Series | int, y: Any) -> tuple:
        return th.tensor(X.values.astype(float) if isinstance(X, pd.Series) else X), \
            th.tensor(y)

class Compose(Transform):
    def __init__(self, transforms: list[Transform]) -> None:
        self.transforms = transforms

    def __call__(self, x: Any, y: Any) -> Any:
        for transform in self.transforms:
            x, y = transform(x, y)
        return x, y

In [37]:
composed_transform = Compose([OrdinalEncoderTransform(category_columns), LabelEncoderTransform(), ToTensor()])
bank_dataset = BankDataset(data, transform=composed_transform)
x, y = bank_dataset[0]
x, y

(tensor([ 5.8000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          2.1430e+03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
          0.0000e+00,  2.6100e+02,  1.0000e+00, -1.0000e+00,  0.0000e+00,
          0.0000e+00], dtype=torch.float64),
 tensor(0))

In [38]:
x, y = bank_dataset[83]
x, y

(tensor([ 5.9000e+01,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          2.3430e+03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
          0.0000e+00,  1.0420e+03,  1.0000e+00, -1.0000e+00,  0.0000e+00,
          0.0000e+00], dtype=torch.float64),
 tensor(1))

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

- [ ] Проверено на семинаре

In [39]:
train, test = random_split(bank_dataset, lengths=[0.75, 0.25])
train_loader = DataLoader(train, 64)

it = iter(train_loader)
x, y = next(it)
x.shape, y.shape

(torch.Size([64, 16]), torch.Size([64]))

In [40]:
def collate_fn(batch):
    x, y = zip(*batch)
    x = th.stack(x).view(-1, 2, 8)
    y = th.tensor(y)
    return x, y

In [41]:
train_loader = DataLoader(train, 64, shuffle=True, collate_fn=collate_fn)
x, y = next(iter(train_loader))
print(x.shape, y.shape)

torch.Size([64, 2, 8]) torch.Size([64])
